In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

In [42]:
#Motley-fool data with transcript for different tickers (company code) and dates
earnings_call_data = pd.read_pickle(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\motley-fool-data.pkl\motley-fool-data.pkl")

#adjusting the date format and aligning with market closing time date
earnings_call_data['date'] = earnings_call_data['date'].str.replace(r'\s*ET$', '', regex=True)
earnings_call_data['date'] = earnings_call_data['date'].str.replace('a.m.', 'AM', regex=False).str.replace('p.m.', 'PM', regex=False).str.strip()
earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])
earnings_call_data['adjusted_date'] = earnings_call_data['date'].apply(lambda x: x + pd.offsets.BDay(1) if x.hour >= 16 else x)
earnings_call_data['adjusted_date'] = earnings_call_data['adjusted_date'].dt.normalize()
earnings_call_data['adjusted_date']=pd.to_datetime(earnings_call_data['adjusted_date'])


C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\2414088161.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  earnings_call_data['date'] = pd.to_datetime(earnings_call_data['date'])


In [44]:
earnings_call_data.head()

,date,exchange,q,ticker,transcript,adjusted_date
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07


In [4]:
#defining start and end date to get the time range for the dataset we have 
start_date=earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date=earnings_call_data['adjusted_date'].max().strftime('%Y-%m-%d')
tickers = earnings_call_data['ticker'].unique().tolist()

In [5]:
#getting the stock prices data from yfinance for the data range above
stock_prices = yf.download(tickers, start=start_date, end=end_date)


[                       0%                       ]  9 of 2876 completed$IMV: possibly delisted; no timezone found
[                       1%                       ]  16 of 2876 completed$INFN: possibly delisted; no timezone found
[                       1%                       ]  17 of 2876 completed$EVRI: possibly delisted; no timezone found
[                       1%                       ]  18 of 2876 completed$LIVX: possibly delisted; no timezone found
[                       1%                       ]  19 of 2876 completed$PRTK: possibly delisted; no timezone found
[                       1%                       ]  19 of 2876 completed$TLIS): possibly delisted; no timezone found
[                       1%                       ]  21 of 2876 completed$DEN: possibly delisted; no timezone found
[                       1%                       ]  22 of 2876 completedHTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: 

In [6]:
#extracting just the stock prices from the data set above
adjusted_close_prices = stock_prices['Close']
adjusted_close_prices.tail()

Ticker,A,AA,AAIC,AAL,AAN,AAOI,AAON,AAP,AAPL,AAT,...,ZLND.Y,ZM,ZNGA,ZS,ZTS,ZUMZ,ZUO,ZVIA,ZY,ZYXI
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-15,148.687668,46.684223,NaN,16.799999,NaN,2.83,51.950039,142.273819,153.255112,22.068243,...,NaN,80.769997,NaN,142.770004,NaN,25.530001,NaN,4.50,NaN,12.97
2023-02-16,145.287750,47.031391,NaN,16.389999,NaN,2.95,51.765827,141.239365,151.656769,21.918570,...,NaN,77.910004,NaN,132.929993,NaN,25.219999,NaN,4.49,NaN,12.89
2023-02-17,145.268112,45.218414,NaN,16.360001,NaN,2.80,52.634289,141.051300,150.512268,21.868683,...,NaN,76.110001,NaN,132.350006,NaN,25.309999,NaN,4.47,NaN,13.05
2023-02-21,140.525772,46.356346,NaN,15.690000,NaN,2.72,51.035519,135.596939,146.496643,21.186842,...,NaN,72.470001,NaN,127.839996,NaN,24.090000,NaN,4.24,NaN,12.61
2023-02-22,139.212814,47.233906,NaN,15.560000,NaN,2.82,51.338169,135.060913,146.920883,21.403034,...,NaN,73.389999,NaN,132.160004,NaN,24.320000,NaN,4.37,NaN,12.54


#### Only SP500

In [24]:
folder =r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\stock_data"
start_date = earnings_call_data['adjusted_date'].min().strftime('%Y-%m-%d')
end_date = "2022-12-12"

dates = pd.date_range(start_date, end_date, freq="D")


In [25]:
import os
sp500_tickers = []
for file in os.listdir(folder):
    ticker = os.path.splitext(file)[0]
    sp500_tickers.append(ticker)
    required_tickers = [t for t in sp500_tickers if t in tickers]
    

In [26]:
series_by_ticker = {} # dictionary to hold ticker prices for each ticker

for t in required_tickers:
    path = os.path.join(folder, f"{t}.csv")
    df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
    adj_price = df[["Date", "Adj Close"]]
    s = adj_price.set_index('Date')["Adj Close"].rename(t)
    series_by_ticker[t] = s


C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\278945525.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\278945525.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df = pd.read_csv(path, parse_dates = ["Date"], infer_datetime_format=True)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\278945525.py:5: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict ve

In [27]:
prices_df = pd.DataFrame(series_by_ticker) #dataframe with all ticker prices


In [28]:
prices_df.index = pd.to_datetime(prices_df.index).normalize()
start = pd.to_datetime(start_date) - pd.tseries.offsets.BDay(20)
end = pd.to_datetime(end_date)

# slice by datetime range
new_df = prices_df.loc[start:end]
# Calculate 1-day, 5-day returns (targets) and 1-day, 5-day and 20-day before returns for each ticker
for ticker in new_df.columns:
    new_df[f'{ticker}_bcallday_r1'] = new_df[ticker].pct_change(1).shift(1)
    new_df[f'{ticker}_bcallday_r5'] = new_df[ticker].pct_change(5).shift(1)
    new_df[f'{ticker}_bcallday_r20'] = new_df[ticker].pct_change(20).shift(1)
    new_df[f'{ticker}_r1d'] = new_df[ticker].shift(-1) / new_df[ticker] - 1
    new_df[f'{ticker}_r5d'] = new_df[ticker].shift(-5) / new_df[ticker] - 1

C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1534260903.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_bcallday_r1'] = new_df[ticker].pct_change(1).shift(1)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1534260903.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df[f'{ticker}_bcallday_r5'] = new_df[ticker].pct_change(5).shift(1)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1534260903.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

In [31]:
new_df.tail()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,ZION_bcallday_r1,ZION_bcallday_r5,ZION_bcallday_r20,ZION_r1d,ZION_r5d,ZTS_bcallday_r1,ZTS_bcallday_r5,ZTS_bcallday_r20,ZTS_r1d,ZTS_r5d
Date,,,,,,,,,,,,,,,,,,,,,
2022-12-06,151.350006,14.33,145.529999,142.910004,163.720001,169.320007,377.950012,103.860001,60.540001,285.410004,...,-0.073301,-0.036034,-0.018998,-0.001909,NaN,-0.013277,0.048606,0.162041,-0.018295,NaN
2022-12-07,153.729996,13.55,146.649994,140.940002,165.399994,170.289993,377.790009,104.809998,60.150002,286.690002,...,-0.026435,-0.076230,-0.047801,0.007014,NaN,-0.014678,0.035171,0.110184,0.022829,NaN
2022-12-08,156.279999,13.60,146.589996,142.649994,165.990005,170.460007,380.750000,106.919998,60.560001,292.149994,...,-0.001909,-0.092049,-0.064612,0.002322,NaN,-0.018295,-0.025237,0.103076,-0.001887,NaN
2022-12-09,152.949997,13.53,145.899994,142.160004,163.059998,165.330002,380.750000,107.510002,60.040001,288.410004,...,0.007014,-0.088671,-0.046872,0.019162,NaN,0.022829,-0.018897,0.154014,-0.003455,NaN
2022-12-12,155.330002,14.20,145.929993,144.490005,165.320007,167.199997,380.010010,109.489998,60.000000,292.339996,...,0.002322,-0.091100,-0.118433,NaN,NaN,-0.001887,-0.025600,0.075591,NaN,NaN


In [32]:
spy_series = yf.download("SPY", start=(start- pd.tseries.offsets.BDay(20)).strftime("%Y-%m-%d") , end=(end + pd.Timedelta(days=1)).strftime("%Y-%m-%d"), progress=False)["Close"]

# normalize index, sort, compute returns
spy_series.index = pd.to_datetime(spy_series.index).normalize()
spy_series = spy_series.sort_index()


In [33]:
spy_series["m_r1d"] = spy_series["SPY"].shift(-1) / spy_series["SPY"] - 1
spy_series["m_r5d"] = spy_series["SPY"].shift(-5) / spy_series["SPY"] - 1
spy_series["m_bcallday_r1"] = spy_series["SPY"].pct_change(1).shift(1)
spy_series["m_bcallday_r5"] = spy_series["SPY"].pct_change(5).shift(1)
spy_series["m_bcallday_r20"] = spy_series["SPY"].pct_change(20).shift(1)

In [34]:
df_with_market = pd.merge(new_df, spy_series, left_index = True, right_index = True ,how = "left")

In [35]:
df_with_market

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACGL,ACN,...,ZTS_bcallday_r5,ZTS_bcallday_r20,ZTS_r1d,ZTS_r5d,SPY,m_r1d,m_r5d,m_bcallday_r1,m_bcallday_r5,m_bcallday_r20
Date,,,,,,,,,,,,,,,,,,,,,
2019-03-14,78.459381,31.704929,146.551392,44.523067,66.664124,74.894989,325.609985,74.421806,32.549999,156.109482,...,NaN,NaN,0.007381,0.035759,252.856750,0.004940,0.017158,0.006619,0.014460,NaN
2019-03-15,78.936333,31.014189,147.350800,45.102238,67.823143,75.526093,333.230011,75.250999,32.869999,157.596481,...,NaN,NaN,0.008874,0.010525,254.105957,0.003626,-0.007322,-0.000640,0.022363,NaN
2019-03-18,78.809799,30.964851,153.755264,45.562660,67.247810,75.723907,333.250000,74.280457,32.840000,157.634354,...,NaN,NaN,0.009717,0.001636,255.027374,0.000248,-0.011653,0.004940,0.029473,0.029773
2019-03-19,79.033638,31.172070,155.515793,45.201588,67.389565,76.364426,339.359985,75.345245,32.360001,157.416519,...,NaN,NaN,-0.002229,0.001013,255.090591,-0.003010,-0.004533,0.003626,0.018437,0.022366
2019-03-20,78.761116,30.550407,156.210449,45.596592,67.064362,74.668900,339.660004,75.326401,31.980000,157.482819,...,NaN,NaN,0.011573,0.000203,254.322693,0.011295,-0.006748,0.000248,0.014863,0.020853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06,151.350006,14.330000,145.529999,142.910004,163.720001,169.320007,377.950012,103.860001,60.540001,285.410004,...,0.048606,0.162041,-0.018295,NaN,376.844604,-0.001701,NaN,-0.017989,0.009295,0.061751
2022-12-07,153.729996,13.550000,146.649994,140.940002,165.399994,170.289993,377.790009,104.809998,60.150002,286.690002,...,0.035171,0.110184,0.022829,NaN,376.203552,0.007834,NaN,-0.014415,-0.003542,0.036531
2022-12-08,156.279999,13.600000,146.589996,142.649994,165.990005,170.460007,380.750000,106.919998,60.560001,292.149994,...,-0.025237,0.103076,-0.001887,NaN,379.150696,-0.007470,NaN,-0.001701,-0.035616,0.029215


In [36]:
for t in required_tickers:
    df_with_market[f'{t}_abn_r1d']=df_with_market[f'{t}_r1d']-df_with_market['m_r1d']
    df_with_market[f'{t}_abn_r5d']=df_with_market[f'{t}_r5d']-df_with_market['m_r5d']
    df_with_market[f'{t}_abn_bcallday_r1']=df_with_market[f'{t}_bcallday_r1']-df_with_market['m_bcallday_r1']
    df_with_market[f'{t}_abn_bcallday_r5']=df_with_market[f'{t}_bcallday_r5']-df_with_market['m_bcallday_r5']
    df_with_market[f'{t}_abn_bcallday_r20']=df_with_market[f'{t}_bcallday_r20']-df_with_market['m_bcallday_r20']
    df_with_market[f'{t}_abn_vol20d'] = df_with_market[f'{t}_abn_r1d'].rolling(window=20).std()
    

C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1588641288.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_with_market[f'{t}_abn_r5d']=df_with_market[f'{t}_r5d']-df_with_market['m_r5d']
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1588641288.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_with_market[f'{t}_abn_bcallday_r1']=df_with_market[f'{t}_bcallday_r1']-df_with_market['m_bcallday_r1']
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\1588641288.py:5: PerformanceWarning: DataFrame is highly fragmented.  

In [45]:
# convert wide df_with_market columns like "TICK_abn_r1d" -> long table and merge
df_with_market.index = pd.to_datetime(df_with_market.index).normalize()
earnings_call_data['adjusted_date'] = pd.to_datetime(earnings_call_data['adjusted_date']).dt.normalize()

# 1d abnormal returns long
abn1 = df_with_market.filter(regex=r'_abn_r1d$').copy()
abn1.columns = [c.rsplit('_abn_r1d', 1)[0] for c in abn1.columns]   # rename cols -> ticker
abn1_long = abn1.stack().rename('abret_1d').reset_index()
abn1_long.columns = ['adjusted_date', 'ticker', 'abret_1d']

# 5d abnormal returns long
abn5 = df_with_market.filter(regex=r'_abn_r5d$').copy()
abn5.columns = [c.rsplit('_abn_r5d', 1)[0] for c in abn5.columns]
abn5_long = abn5.stack().rename('abret_5d').reset_index()
abn5_long.columns = ['adjusted_date', 'ticker', 'abret_5d']

# 20-day abnormal volatility long
abnvol20 = df_with_market.filter(regex=r'_abn_vol20d$').copy()
abnvol20.columns = [c.rsplit('_abn_vol20d', 1)[0] for c in abnvol20.columns]
abnvol20_long = abnvol20.stack().rename('abvol_20d').reset_index()
abnvol20_long.columns = ['adjusted_date', 'ticker', 'abvol_20d']    

# abnormal before call day 1-day returns long
abncallday = df_with_market.filter(regex=r'_abn_bcallday_r1$').copy()
abncallday.columns = [c.rsplit('_abn_bcallday_r1', 1)[0] for c in abncallday.columns]
abncallday_long = abncallday.stack().rename('abcallday_r1').reset_index()
abncallday_long.columns = ['adjusted_date', 'ticker', 'abcallday_r1']

#abnormal before call day 5-day returns long
abncallday5 = df_with_market.filter(regex=r'_abn_bcallday_r5$').copy()
abncallday5.columns = [c.rsplit('_abn_bcallday_r5', 1)[0] for c in abncallday5.columns]
abncallday5_long = abncallday5.stack().rename('abcallday_r5').reset_index()
abncallday5_long.columns = ['adjusted_date', 'ticker', 'abcallday_r5']

#abnormal before call day 20-day returns long
abncallday20 = df_with_market.filter(regex=r'_abn_bcallday_r20$').copy()
abncallday20.columns = [c.rsplit('_abn_bcallday_r20', 1)[0] for c in abncallday20.columns]
abncallday20_long = abncallday20.stack().rename('abcallday_r20').reset_index()
abncallday20_long.columns = ['adjusted_date', 'ticker', 'abcallday_r20']

# merge both into earnings_call_data
earnings_call_data = earnings_call_data.merge(
    abn1_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abn5_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abnvol20_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abncallday_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abncallday5_long, on=['adjusted_date', 'ticker'], how='left'
).merge(
    abncallday20_long, on=['adjusted_date', 'ticker'], how='left'
)

In [46]:
earnings_call_data

,date,exchange,q,ticker,transcript,adjusted_date,abret_1d,abret_5d,abvol_20d,abcallday_r1,abcallday_r5,abcallday_r20
0,2020-08-27 21:00:00,NASDAQ: BILI,2020-Q2,BILI,"Prepared Remarks:\nOperator\nGood day, and wel...",2020-08-28,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-07-30 16:30:00,NYSE: GFF,2020-Q3,GFF,Prepared Remarks:\nOperator\nThank you for sta...,2020-07-31,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-10-23 17:00:00,NASDAQ: LRCX,2020-Q1,LRCX,Prepared Remarks:\nOperator\nGood day and welc...,2019-10-24,0.012659,0.010627,0.032239,-0.010274,-0.01169,-0.037597
3,2019-11-06 12:00:00,NASDAQ: BBSI,2019-Q3,BBSI,"Prepared Remarks:\nOperator\nGood day, everyon...",2019-11-06,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-08-07 08:30:00,NASDAQ: CSTE,2019-Q2,CSTE,Prepared Remarks:\nOperator\nGreetings and wel...,2019-08-07,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
18750,2021-11-09 13:00:00,NYSE: SWX,2021-Q3,SWX,Prepared Remarks:\nOperator\nLadies and gentle...,2021-11-09,NaN,NaN,NaN,NaN,NaN,NaN
18751,2021-11-18 12:00:00,NYSE: PNNT,2021-Q4,PNNT,"Prepared Remarks:\nOperator\nGood morning, and...",2021-11-18,NaN,NaN,NaN,NaN,NaN,NaN
18752,2022-02-08 11:00:00,NYSE: TDG,2022-Q1,TDG,Prepared Remarks:\nOperator\nThank you for sta...,2022-02-08,-0.007776,0.026821,0.010281,0.025174,0.03356,0.008346
18753,2022-02-28 16:30:00,NASDAQ: DVAX,2021-Q4,DVAX,"Prepared Remarks:\nOperator\nGood day, ladies ...",2022-03-01,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
clean_df=earnings_call_data.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3544 entries, 2 to 18752
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           3544 non-null   datetime64[ns]
 1   exchange       3544 non-null   object        
 2   q              3544 non-null   object        
 3   ticker         3544 non-null   object        
 4   transcript     3544 non-null   object        
 5   adjusted_date  3544 non-null   datetime64[ns]
 6   abret_1d       3544 non-null   object        
 7   abret_5d       3544 non-null   object        
 8   abvol_20d      3544 non-null   float64       
 9   abcallday_r1   3544 non-null   float64       
 10  abcallday_r5   3544 non-null   float64       
 11  abcallday_r20  3544 non-null   float64       
dtypes: datetime64[ns](2), float64(4), object(6)
memory usage: 359.9+ KB


In [48]:
clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)

C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\443728299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r1d_direction'] = (clean_df['abret_1d'] > 0).astype(int)
C:\Users\ruchi\AppData\Local\Temp\ipykernel_38212\443728299.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['r5d_direction'] = (clean_df['abret_5d'] > 0).astype(int)


In [50]:
final_df = clean_df[["adjusted_date" , "ticker", "transcript", "abvol_20d", "abcallday_r1","abcallday_r5","abcallday_r20","abret_1d" , "abret_5d","r1d_direction","r5d_direction"]]

In [51]:
final_df.to_pickle(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\model_input_data.pkl")

In [52]:
final_df.to_csv(r"C:\Users\ruchi\OneDrive\Ruchira-MAIN\Work\Stats_ML\model_input_data_v2.csv")

In [53]:
final_df["transcript"].iloc[0]

"Prepared Remarks:\nOperator\nGood day and welcome to the September 2019 Quarter Financial Call for Lam Research. At this time, I'd like to turn the conference over to Ms. Tina Correia, Corporate Vice President of Investor Relations. Please go ahead ma'am.\nTina Correia -- Vice President, Investor Relations & Corporate Communications\nThank you, operator. Thank you and good afternoon everyone, welcome to the Lam Research Quarterly Earnings Conference Call. With me today are: Tim Archer, President and Chief Executive Officer and Doug Bettinger, Executive Vice President and Chief Financial Officer.\nDuring today's call, we will share our overview on the business environment and review our financial results for the September 2019 quarter and our outlook for the December 2019 quarter. The press release detailing our financial results was distributed a little after 1 o'clock PM Pacific Time this afternoon. The release can also be found on the Investor Relations section of the Company's webs